In [312]:
import numpy as np
import gymnasium as gym
import random
import matplotlib.pyplot as plt
from collections import deque
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [313]:
env = gym.make('CliffWalking-v0')
state_size = env.observation_space.n
action_size = env.action_space.n

In [314]:
state_size

np.int64(48)

In [315]:
action_size

np.int64(4)

In [316]:
class DQN(nn.Module):
    
    def __init__(self,state_size,action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size,24)
        self.fc2 = nn.Linear(24,24)
        self.fc3 = nn.Linear(24,action_size)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [317]:
dqn = DQN(state_size,action_size)

In [318]:
x = torch.ones((1,48))

In [319]:
x

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [320]:
dqn(x)

tensor([[0.0214, 0.0153, 0.1408, 0.0546]], grad_fn=<AddmmBackward0>)

In [321]:
np.random.random()

0.5669245214256547

In [322]:
def convert(state):
    ans = np.zeros(state_size)
    ans[state] = 1
    state_tensor = torch.FloatTensor(ans)
    return state_tensor

In [323]:
class Agent:
    
    def __init__(self):
        
        self.eplison = 1.0
        self.eplison_min = 0.01
        self.eplison_decay = 0.995
        self.gamma = 0.95
        
        self.model = DQN(state_size,action_size)
        self.crition = nn.MSELoss()
        self.optimizer = optim.AdamW(self.model.parameters(),lr=0.001)
        
        self.memory = deque(maxlen=2000)
    
    
    def take_action(self,state,explore=True):
        if explore:
            if np.random.random() < self.eplison:
                action = np.random.randint(action_size)
            else:
                with torch.no_grad():
                    action_values = self.model(state)
                    action = torch.argmax(action_values).item()
            return action
        else:
            with torch.no_grad():
                action_values = self.model(state)
                action = torch.argmax(action_values).item()
            return action
                

    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
    
    
    def update(self):
        batch_size = 32
        arr = random.sample(self.memory,batch_size)
        loss = 0
        for state,action,reward,next_state,done in arr:
            
            state_tensor = convert(state)
            action_values = self.model(state_tensor)
            
            next_state_tensor = convert(next_state)
            target_f = reward
            if not done:
                with torch.no_grad():
                    target_values = self.model(next_state_tensor)
                    target_f = reward + self.gamma * torch.max(target_values)
                    target_f = target_f.item()
#             tag = action_values.clone().detach()
#             tag[action] = target_f
            loss += (action_values[action]-target_f)**2
        loss /= len(arr)
#         loss = self.crition(action_values,tag)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        if self.eplison > self.eplison_min:
            self.eplison *= self.eplison_decay

In [324]:
agent = Agent()

In [325]:
for e in range(500):
    
    state,_ = env.reset()
    count = 0
    rw = 0
    for x in range(500):
        state_tensor = convert(state)
        action = agent.take_action(state_tensor)
        next_state,reward,done,_,_ = env.step(action)
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        count += 1
        rw += reward
        if done:
            break
        
        if len(agent.memory) > 32:
            agent.update()
        

    if e % 10 == 0:  # 修正条件
        print(f"Episode: {e}/{500}, Reward: {rw}, Epsilon: {agent.eplison:.4f}")


Episode: 0/500, Reward: -891, Epsilon: 0.4373
Episode: 10/500, Reward: -500, Epsilon: 0.0100
Episode: 20/500, Reward: -17, Epsilon: 0.0100
Episode: 30/500, Reward: -17, Epsilon: 0.0100
Episode: 40/500, Reward: -15, Epsilon: 0.0100
Episode: 50/500, Reward: -15, Epsilon: 0.0100
Episode: 60/500, Reward: -51, Epsilon: 0.0100
Episode: 70/500, Reward: -15, Epsilon: 0.0100
Episode: 80/500, Reward: -15, Epsilon: 0.0100
Episode: 90/500, Reward: -15, Epsilon: 0.0100
Episode: 100/500, Reward: -15, Epsilon: 0.0100
Episode: 110/500, Reward: -34, Epsilon: 0.0100
Episode: 120/500, Reward: -15, Epsilon: 0.0100
Episode: 130/500, Reward: -500, Epsilon: 0.0100
Episode: 140/500, Reward: -15, Epsilon: 0.0100
Episode: 150/500, Reward: -500, Epsilon: 0.0100
Episode: 160/500, Reward: -15, Epsilon: 0.0100
Episode: 170/500, Reward: -15, Epsilon: 0.0100
Episode: 180/500, Reward: -15, Epsilon: 0.0100
Episode: 190/500, Reward: -15, Epsilon: 0.0100
Episode: 200/500, Reward: -15, Epsilon: 0.0100
Episode: 210/500, Re

In [328]:
def test_agent(agent, episodes=10, render=True):
    env = gym.make('CliffWalking-v0', render_mode='human' if render else None)
    state_size = env.observation_space.n
    
    for e in range(episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            if render:
                env.render()
            state = convert(state)
            action = agent.take_action(state, explore=False)  # 测试时不探索
            next_state, reward, done, _, _ = env.step(action)
            state = next_state
            total_reward += reward
            
            if done:
                print(f"Test episode: {e+1}/{episodes}, score: {total_reward}")
                break

In [329]:
test_agent(agent)

Test episode: 1/10, score: -15



KeyboardInterrupt

